In [38]:
import os

import pandas as pd
import mlflow
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, 
    SplineTransformer, 
    QuantileTransformer, 
    RobustScaler,
    PolynomialFeatures,
    KBinsDiscretizer,
)

TABLE_NAME = "users_churn"# таблица с данными

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "churn" # напишите название вашего эксперимента
RUN_NAME = "preprocessing" 
REGISTRY_MODEL_NAME = "churn_model_alexandervasilev"# название зарегистрированной модели 



In [39]:
import os, psycopg
from dotenv import load_dotenv
load_dotenv()

connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}

connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

df[:2]

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,2043,7361-YPXFS,2017-10-01,NaT,Month-to-month,No,Bank transfer (automatic),64.45,1867.6,DSL,...,Yes,Yes,No,No,Female,1,No,No,Yes,0
1,2044,6557-BZXLQ,2018-10-01,NaT,Month-to-month,No,Electronic check,69.65,1043.3,Fiber optic,...,No,No,No,No,Male,1,No,No,No,0


In [40]:
from sklearn.impute import SimpleImputer

num_columns = ["monthly_charges", "total_charges"]
cat_columns = ["type", "payment_method", "internet_service", "gender"]

df[num_columns] = SimpleImputer(strategy='mean').fit_transform(df[num_columns])

n_knots = 3
degree_spline = 4
n_quantiles = 100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None

encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline)
encoder_q = QuantileTransformer(n_quantiles=n_quantiles)
encoder_rb = RobustScaler()
encoder_pol = PolynomialFeatures(degree=degree)
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample)

encoder_oh = OneHotEncoder(categories='auto', handle_unknown='ignore', max_categories=10, sparse_output=False, drop='first')

In [43]:
numeric_transformer = ColumnTransformer(
    transformers=[
        ('spl', encoder_spl, num_columns),
        ('q', encoder_q, num_columns),
        ('rb', encoder_rb, num_columns),
        ('pol', encoder_pol, num_columns),
        ('kbd', encoder_kbd, num_columns)
    ]
)

categorical_transformer = Pipeline(steps=[('encoder', encoder_oh)])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_columns),
        ('cat', categorical_transformer, cat_columns)
    ], n_jobs=-1
)

encoded_features = preprocessor.fit_transform(df)
transformed_df = pd.DataFrame(encoded_features, columns=preprocessor.get_feature_names_out())

#transformed_df.head(10)
new_df = pd.concat([df, transformed_df], axis=1)
pd.set_option('display.max_columns', None)  # Отображать все колонки
new_df.head(10)

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target,num__spl__monthly_charges_sp_0,num__spl__monthly_charges_sp_1,num__spl__monthly_charges_sp_2,num__spl__monthly_charges_sp_3,num__spl__monthly_charges_sp_4,num__spl__monthly_charges_sp_5,num__spl__total_charges_sp_0,num__spl__total_charges_sp_1,num__spl__total_charges_sp_2,num__spl__total_charges_sp_3,num__spl__total_charges_sp_4,num__spl__total_charges_sp_5,num__q__monthly_charges,num__q__total_charges,num__rb__monthly_charges,num__rb__total_charges,num__pol__1,num__pol__monthly_charges,num__pol__total_charges,num__pol__monthly_charges^2,num__pol__monthly_charges total_charges,num__pol__total_charges^2,num__pol__monthly_charges^3,num__pol__monthly_charges^2 total_charges,num__pol__monthly_charges total_charges^2,num__pol__total_charges^3,num__kbd__monthly_charges,num__kbd__total_charges,cat__type_One year,cat__type_Two year,cat__payment_method_Credit card (automatic),cat__payment_method_Electronic check,cat__payment_method_Mailed check,cat__internet_service_Fiber optic,cat__internet_service_None,cat__gender_Male
0,2043,7361-YPXFS,2017-10-01,NaT,Month-to-month,No,Bank transfer (automatic),64.45,1867.60,DSL,No,Yes,Yes,Yes,No,No,Female,1,No,No,Yes,0,0.000002,0.056804,0.496757,0.416666,2.977230e-02,0.000000e+00,0.004502,0.232796,0.595606,0.165716,1.381000e-03,0.000000,0.440187,0.577099,-0.108556,0.138002,1.0,64.45,1867.60,4153.8025,120366.820,3.487930e+06,2.677126e+05,7.757642e+06,2.247971e+08,6.514058e+09,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2044,6557-BZXLQ,2018-10-01,NaT,Month-to-month,No,Electronic check,69.65,1043.30,Fiber optic,No,No,No,No,No,No,Male,1,No,No,No,0,0.000000,0.037981,0.446766,0.469639,4.561382e-02,1.142973e-08,0.014163,0.332225,0.556750,0.096732,1.302212e-04,0.000000,0.482560,0.420820,-0.012879,-0.105559,1.0,69.65,1043.30,4851.1225,72665.845,1.088475e+06,3.378807e+05,5.061176e+06,7.581228e+07,1.135606e+09,2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
2,2045,2550-QHZGP,2019-07-01,NaT,One year,No,Mailed check,19.50,128.60,None,None,None,None,None,None,None,Male,0,No,No,No,0,0.037673,0.445748,0.470609,0.045970,1.595455e-08,0.000000e+00,0.037601,0.445511,0.470835,0.046053,1.718080e-08,0.000000,0.030303,0.128695,-0.935603,-0.375830,1.0,19.50,128.60,380.2500,2507.700,1.653796e+04,7.414875e+03,4.890015e+04,3.224902e+05,2.126782e+06,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
3,2046,7519-JTWQH,2014-05-01,NaT,Two year,Yes,Bank transfer (automatic),110.50,7455.45,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Female,0,No,No,Yes,0,0.000000,0.000030,0.076385,0.531653,3.715967e-01,2.033488e-02,0.000000,0.000270,0.111803,0.570269,3.066895e-01,0.010968,0.972832,0.971421,0.738730,1.789075,1.0,110.50,7455.45,12210.2500,823827.225,5.558373e+07,1.349233e+06,9.103291e+07,6.142003e+09,4.144018e+11,4.0,4.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2047,2538-OIMXF,2020-01-01,NaT,Month-to-month,No,Mailed check,24.70,24.70,DSL,No,No,No,No,No,No,Female,0,No,Yes,None,0,0.024051,0.391047,0.517404,0.067486,1.131054e-05,0.000000e+00,0.041440,0.457652,0.459014,0.041894,1.432327e-13,0.000000,0.189033,0.023407,-0.839926,-0.406530,1.0,24.70,24.70,610.0900,610.090,6.100900e+02,1.506922e+04,1.506922e+04,1.506922e+04,1.506922e+04,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,2048,8543-MSDMF,2019-11-01,NaT,Month-to-month,Yes,Electronic check,77.40,206.15,DSL,Yes,No,Yes,No,Yes,Yes,Male,0,No,No,No,0,0.000000,0.019105,0.364548,0.536516,7.979011e-02,4.100195e-05,0.034914,0.436287,0.479445,0.049353,1.456293e-07,0.000000,0.584610,0.169597,0.129715,-0.352916,1.0,77.40,206.15,5990.7600,15956.010,4.249782e+04,4.636848e+05,1.234995e+06,3.289331e+06,8.760926e+06,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
6,2049,9961-JBNMK,2018-03-01,2019-12-01,Month-to-month,Yes,Bank transfer (automatic),96.80,2030.30,Fiber optic,No,No,Ye

In [35]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    mlflow.sklearn.log_model(preprocessor, "column_transformer") 

2025/09/13 10:40:13 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [36]:
run_id

'5005dc7a5b3a40fdbea5078f91d5ee25'